![](./img/chinahadoop.png)
# 互联网金融风控面试业务知识手册
**[小象学院](http://www.chinahadoop.cn/course/landpage/15)《机器学习集训营》课程资料 by [@寒小阳](https://www.chinahadoop.cn/user/49339/about)**

采集自互联网金融风控从业人员与面试官&&知乎

#### 互联网金融场景下的的风控模型种类？

- 获客阶段：用户响应模型，风险预筛选模型。
- 授信阶段：申请评分模型，反欺诈模型，风险定价模型，收益评分模型。
- 贷后阶段：行为评分模型，交易欺诈模型，客户流失模型。
- 催收阶段：早期催收模型，晚期催收模型。



#### 简单描述一下风控建模的流程？

- 前期准备工作：不同的模型针对不同的业务场景，在建模项目开始前需要对业务的逻辑和需求有清晰的理解，明确好模型的作用，项目周期时间和安排进度，以及模型效果的要求。
- 模型设计：包括模型的选择（评分卡还是集成模型），单个模型还是做模型的细分，是否需要做拒绝推论，观察期，表现期的定义，好坏用户的定义，数据的获取途径等都要确定好。
- 数据拉取及清洗：根据观察期和表现期的定义从数据池中取数，并进行前期的数据清洗和稳定性验证工作，数据清洗包括用户唯一性检查，缺失值检查，异常值检查等。稳定性验证主要考察变量在时间序列上的稳定性，衡量的指标有PSI，平均值/方差，IV等。
- 特征工程：主要做特征的预处理和筛选，如果是评分卡，需要对特征进行离散化，归一化等处理，再对特征进行降维，降维的方法有IV筛选，相关性筛选，显著性筛选等。另外会基于对业务的深入理解做特征构造工作，包括特征交叉，特征转换，对特征进行四则运算等。
- 模型建立和评估：选择合适的模型，像评分卡用逻辑回归，只需要做出二分类预测可以选择xgboost等集成模型，模型建好后需要做模型评估，计算AUC,KS，并对模型做交叉验证来评估泛化能力及模型的稳定性。
- 模型上线部署：在风控后台上配置模型规则，对于一些复杂的模型还得需要将模型文件进行转换，并封装成一个类，用Java等其他形式来调用。
- 模型监控：前期主要监控模型整体及变量的稳定性，衡量标准主要是PSI，并每日观察模型规则的拒绝率与线下的差异。后期积累一定线上用户后可评估线上模型的AUC,KS，与线下进行比较，衡量模型的线上的实际效果。



#### 评分卡，集成模型在线上是如何部署的？

- 评分卡的部署较为简单，因为评分卡将变量映射到了一个个区间及得分，所以在普通的风控决策引擎上就可配置。
- 像一些比较复杂的模型，例如xgboost和lightgbm，一般是将模型文件转换为pmml格式，并封装pmml，在风控后台上上传pmml文件和变量参数文件，并配置好模型的阈值。python模型和R模型都可以用这种方式来部署。



#### 对于金融场景，稳定胜于一切，那在建模过程中如何保证模型的稳定性？

- 在数据预处理阶段可以验证变量在时间序列上的稳定性，通过这个方法筛掉稳定性不好的变量，也能达到降维的目的。筛选的手段主要有：计算月IV的差异，观察变量覆盖率的变化，两个时间点的PSI差异等。
- 异常值的检查，剔除噪声，尤其对于逻辑回归这种对于噪声比较敏感的模型。
- 在变量筛选阶段剔除与业务理解相悖的变量，如果是评分卡，可以剔除区分度过强的变量，这种变量一般不适合放入模型中，否则会造成整个模型被这个变量所左右，造成模型的稳定性下降，过拟合的风险也会增加。
- 做交叉验证，一种是时间序列上的交叉验证，考察模型在时间上的稳定性，另一种是K折随机交叉验证，考察模型的随机稳定性。
- 选择稳定性较好的模型，例如随机森林或xgboost这类泛化能力较好的模型。



#### 为什么要做拒绝推断？常用的拒绝推断方法有哪些？

拒绝推断的目的：

- 如果只用好坏用户建模，则忽略了那些授信被拒的用户，加入拒绝用户是为了让建模样本更接近总体的分布，防止样本有偏，同时也能增加样本数量。
- 公司内部策略的变动，导致当前的申请者已不能代表建模时点的申请者，所以过去被拒的用户不一定现在也会被拒绝，因此，只使用审批通过的用户可能会造成误判。
- 做拒绝推断可以找出之前被拒的好用户，挖掘这些用户，改善风控流程，增加公司收益。

拒绝推断的适用范围：

高核准率不适合用拒绝推断，因为高核准率下好坏用户已接近于整体的申请用户。中低核准率适用用拒绝推断。

拒绝推断的常用方法：

- 硬性截断法：先用好坏用户建立初始模型，然后用这个初始模型对拒绝用户进行打分，设定一个阈值分数（根据对拒绝用户的风险容忍度），低于这个阈值的为坏用户，高于这个阈值的为好用户。再将已标记好的拒绝用户放入样本中，重新建立模型。
- 分配法：此方法适用于评分卡，先用好坏用户建立初始评分卡模型，再将样本跟据评分高低进行分组，计算各分组的违约率。然后对拒绝用户进行打分并按此前的步骤进行分组，以各分组的违约率为抽样比例，随机抽取改分组下的违约用户，指定其为坏用户，剩下的则是好用户。最后将已标记的拒绝用户加入样本中，重新建立模型。
- 平常工作中主要用到以上两种方法，个人建议做申请模型最好做一下拒绝推断，这样模型上线后的得分分布和拒绝率和线下才不会有很大的差异。



#### 模型转化为规则后决策点（cutoff点）怎么设定？

- 规则只是判断用户好坏，而不会像模型会输出违约概率，所以设定决策点时需要考虑到规则的评估指标（精准率，查全率，误伤率，拒绝率），一般模型开发前会设定一个预期的拒绝率，在这个拒绝率下再考量精确率，查全率和误伤率的取舍，找到最佳的平衡点。
- 好的模型能接受更多的好用户，拒绝掉更多的坏用户，也就是提高好坏件比例，所以可事先设定一个预期目标的好坏件比例来选择最佳的决策点。



#### 怎么做风控模型的冷启动？

风控模型的冷启动是指产品刚上线时，没有积累的用户数据，或者用户还没有表现出好坏，此时需要做模型就是一个棘手的问题，常用的方法如下：

- 不做模型，只做规则。凭借自己的业务经验，做一些硬性规则，比如设定用户的准入门槛，考量用户的信用历史和多头风险，而且可以接入第三方提供的反欺诈服务和数据产品的规则。另外可以结合人审来对用户的申请资料做风险评估。
- 借助相同模式产品的数据来建模。如果两个产品的获客渠道，风控逻辑，用户特征都差不多的话，可以选择之前已上线那个产品所积累的用户来建模，不过在模型上线后需要比较线上用户的特征是否与建模用户有较大的差异，如果差异较大，需要对模型对一些调整。
- 无监督模型+评分卡。这种方法适用于产品上线一段时间后，表现出好坏的用户比较少，但需要做一个模型出来，此时可用线上的申请用户做无监督模型，找出一部分坏样本和好样本，用这些数据来做评分卡模型，当然这种模型准确性是存疑的，需要后续对模型不断迭代优化。



#### 模型上线后是怎么监控的？

前期监控（模型上线后一个月内）：

- 模型最后设定cutoff点后可以得出模型的拒绝率（线下拒绝率）, 上线后需要比较模型每日的拒绝率与线下拒绝率。如果两者差异较大，说明线上的用户与建模的用户分布有很大差异，原因可能是没做拒绝推断，或者用户属性随着时间发生了偏移。
- 监控模型整体的稳定性，通常用PSI来衡量两个时间点的差异程度。模型的稳定性是一个需要长期观察的指标，可绘制月/周PSI变化趋势图来分析稳定性的变化，从中可以发现用户是否随着时间推移属性发生了变化，以便及时对模型做出合理的调整。
- 变量稳定度分析，目的是如果模型的稳定性不好，可利用变量稳定度分析来了解是哪些变量造成的。对于不稳定的变量要分析其原因，并对模型做出调整，弃用不稳定的变量或者找其他变量来替换。

后期监控（用户表现出了好坏程度）：

- 此时已积累了一些线上的好坏用户，可做模型的线上效果的评估，评估的指标有AUC, KS, 基尼系数，如果模型的线下效果好，但线上效果却不理想，这个模型是要做优化的。
- 好坏用户的评分分布。绘制线上好坏用户的评分分布图，如果符合期望（高分段好用户占比多，低分段坏用户占比多），则说明模型的线上的区隔能力较好。
- 变量鉴别力分析。用线上的好坏用户来计算变量的IV值，评价变量的预测能力，预测能力不好的变量可以考虑弃用。



#### 怎么设计反欺诈模型？

本人之前没做过风控的反欺诈模型，因为公司的反欺诈检测都是外包给第三方的，所以对于如何设计反欺诈模型只能给出自己的一些见解：

- 反欺诈模型不太适合用二分类监督模型来做
    - 一是诈骗的类型很多，是一个多分类的问题，不可能只做单一类型的反欺诈模型。
    - 二是大部分数据都是没有标签的，各种监督学习模型几乎无用武之地，而且区分噪声和异常点难度很大，甚至需要一点点想象力和直觉。三是欺诈类型不断在变化，如果用历史数据建的模型来识别之前从未出现过的欺诈类型，几乎是做不到的。
- 个人觉得做模型之前需要确定两个问题，一是如何发现欺诈点，二是如何验证欺诈点。第一个问题可以用数据可视化的手段，做一下时序分析，或者用无监督学习来识别异常点。第二个问题可以用统计学的方法来验证异常点与总体是有显著性差异的，但有差异不一定就是欺诈，所以需要与领域专家进行讨论，也可以直接叫来领域专家来分析用户哪些行为是欺诈的。
- 反欺诈规则+机器学习模型来检测欺诈用户，不能单纯依靠机器学习模型来检测欺诈。规则和模型用到的变量一般有：用户的设备，位置信息，关系网络，异常操作记录和第三方黑名单等。总之反欺诈模型难度很高，有相关经验的小伙伴可以互相交流一下。



#### 当模型上线后发现稳定性不佳，或者线上的区分效果不好，你是怎么对模型作调整的？

- 模型稳定性不佳先检查当初建模时有没有考量过特征的稳定性，在模型前期监控一般会做变量的稳定性分析，如果发现稳定性不佳的变量，考虑弃用或用其他变量替代。另外可以分析下线上用户和建模用户的分布差异，考虑在建模时增加拒绝推断的步骤，让建模样本的分布更加接近于实际整体的申请用户。
- 线上的效果不好可以从变量角度分析，做一下变量鉴别度分析，剔除掉效果不好的变量，挖掘新的变量入模。如果一个模型已上线较长的时间，用户的属性也慢慢发生偏移，建议重新取数做一个新的模型替代旧模型。



#### 如何衡量一个风控模型的效果？

本人在工作中写过一个评分卡模型的评估方法，这里贴出来可以做个参考：

- 1.评分卡建模之前的评估：
    - 主要评估建模样本的稳定性，根据评分卡的目的不同，比较对象为总体或者近段时间的样本。

- 2.分箱过程的评估
    - 变量分箱的同时会计算WOE，这里是对WOE进行可解释性上的评估，包括变化趋势，箱体之间WOE差异，WOE绝对值大小等。

- 3.对逻辑回归模型的评估
    - 将数据集随机划分为训练集和测试集，计算AUC, KS及Gini系数
    - 通过交叉验证的方法，评估模型的泛化能力，评判指标选择AUC。
    - 绘制学习曲线，评估模型是否有过拟合的风险，评判指标为准确率（Accuracy）。

- 4.转化评分之后的评估
    - 对score进行可解释上的评估，评估原则与WOE评估大致相同。
    - 绘制评分分布图，观察分布的形状及好坏用户分布的重叠程度。
    - 绘制提升图和洛伦兹曲线，评估评分卡的可解释性和好坏用户区分效果。
    - 评估准确性，根据对精确率和查全率的重视程度绘制PR曲线，并根据业务目标设定cutoff点。

- 5.评分卡上线后的评估
    - 绘制评分分布表和评分分布图，计算评分的PSI，评估其稳定性。
    - 评估每个入模变量的稳定性。



#### 在实际应用中，如何权衡模型的性能，可解释性和部署的难易程度？

首先要考虑到部署的难易程度，评分卡可以像普通规则一样在风控后台部署，但是像xgboost等比较复杂的模型需要考虑平台支不支持，一般来说能用评分卡解决的就最好用评分卡，部署简单而且可解释性好。然后关于可解释性和模型效果的权衡，个人认为模型的效果达到要求的情况下再去考虑可解释性，如果用评分卡做出来的效果不好，则可以考虑用集成模型或者神经网络等复杂的机器学习模型，只要你的风控后台支持部署。



#### 对于高维稀疏特征，或者是弱特征，你是怎么处理的？

- 对于高维稀疏特征，逻辑回归的效果要比GBDT好。这是由于逻辑回归的正则项是对特征权重的惩罚，以至于特征的权重不至于过大，而树模型的惩罚项主要是深度和叶子节点数目，而对于高维稀疏特征,10000个样本可能9990个值是0，那只需要一个节点就可以划分9990和剩下的10个样本，可见惩罚项之小，所以GBDT对于高维稀疏特征很容易过拟合。平时工作中如果用的是逻辑回归评分卡，则可以对稀疏特征进行离散化，离散成值为0或不为0，再用woe进行编码。而如果使用xgboost等集成模型，最好还是不要用高维的稀疏特征。
- 弱特征指的是与目标变量关系不大的特征，或者是区分能力较弱的特征。在大数据风控中弱特征的种类很多，包括社交，通话，位置等信息，而且建模时弱特征会多达数百个。如果是用评分卡建模，弱特征一般会被舍弃掉，因为评分卡的入模特征数不宜过多，一般在15个以下，所以要找寻比较强的特征。而对于xgboost等模型，本身对数据的要求不是很高，并且精度好，一些弱特征进行交叉组合或许能给模型带来不错的效果。



#### 对于成千上万维的特征你是怎么做特征筛选的，如何保证其模型的可解释性和稳定性？

- 可先做特征的粗筛选，例如缺失率高，方差为0，非常稀疏的特征可以先剔除。
- 根据变量的稳定性再次进行粗筛，衡量指标有月IV差异，两个时间点的PSI差异等。
- 根据IV值的高低筛选变量，或者直接用集成模型的特征重要性进行筛选。
- 为了保证模型的可解释性，需要将共线性的特征剔除。
- 最后考察各个特征与目标变量的关系，要求在业务上有良好的可解释能力，并且特征与目标变量的关系最好是呈单调线性变化的，这样也能保证模型的稳定性。



#### 如何根据风险因素对用户分层，构建客群差异化的模型？

做客群差异化模型之前最好做一下用户画像，在风控领域中做用户画像的目的是：

- 系统性的梳理用户群体，找到异同点对用户进行划分群体，分类的维度很多，可以是静态属性，购买偏好，也可以是褥羊毛党等风险属性。
- 便于更深刻的理解业务，理解用户需求，风控离不开业务，只有深刻理解业务后，才能发现更多潜在的风险。
- 便于后续的数据挖掘，了解坏用户的行为特征，并且根据用户特征做关联规则分析。
- 对不同类型的用户，做针对性的风控规则和风控模型。

平常工作中的做法：

- 对用户做静态属性的划分，比如按性别，年龄，收入，职业等。例如刚毕业工作的年轻人和收入比较稳定的中年人，他们的借款需求，风险程度就不一样，可以先对用户群体做这样的划分，再对每个群体单独建立模型。
- 根据用户风险属性做差异化模型，例如对手机分期业务做一个套现风险模型，挖掘套现风险属性，目标变量变成是否为套现用户。



#### 额度，利率的风险定价模型你是如何设计的？

- 首先做风险定价模型需要熟悉产品的属性和特点，像小额现金贷和大额分期贷两种产品的额度定价逻辑就不同。另外也要了解产品的盈利模式和预期的利润，这点需要与业务部门做好沟通，通常关于额度，利率也是业务或者产品制定的。
- 风险定价模型一般采用评分卡模型，最后设定cutoff点后对通过的用户进行风险等级划分，对于风险高的用户给的额度较低，或者利率较高。一般来说中低额度的用户占大部分，高额度用户占小部分，最后可以得出一个平均额度或利率，这个值事先可以根据预期的利润/资损来计算。



#### 风控流程中不同环节的评分卡是怎么设计的？

- 申请评分A卡用在贷前审核阶段，主要的作用是决定用户是否准入和对用户进行风险定价（确定额度和利率），用到的数据是用户以往的信用历史，多头借贷，消费记录等信息，并且做A卡一般需要做拒绝推断。A卡一般预测用户的首笔借款是否逾期，或者预测一段时间内是否会逾期，设计的方式也多种多样，有风险差异化评分卡，群体差异化评分卡，或者做交叉评分卡等。
- 行为B卡主要用在借贷周期较长的产品上，例如手机分期。作用一是防控贷中风险，二是对用户的额度做一个调整。用到的数据主要是用户在本平台的登录，浏览，消费行为数据，还有借还款，逾期等借贷表现数据。
- 催收C卡主要是对逾期用户做一个画像分析，通过深度挖掘用户特征，对逾期用户进行分群，做智能催收策略等。